## Why use DL when already getting 95-96% accuracy in ML models?

In the classical machine learning models trained earlier, we used the expert-engineered features. But what if we don't have a domain expert available. In this notebook, we have used deep learning (LSTM) on the raw time-series data.

In [1]:
# Importing Libraries
import pandas as pd
import numpy as np

In [2]:
# Activities are the class labels
# It is a 6 class classification problem
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [3]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [4]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [5]:

def load_y(subset):
    """
    The objective is that we are trying to predict is an integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [6]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [8]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

In [13]:
# Configuring a session
session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [20]:
# Import Keras
from keras import backend as K
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

In [21]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [22]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [23]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [24]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [25]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


In [27]:
X_test.shape

(2947, 128, 9)

- Defining the Architecture of LSTM

In [29]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [35]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.1659 - accuracy: 0.9474 - val_loss: 0.4735 - val_accuracy: 0.8935
Epoch 2/30
7352/7352 [==============================] - 29s 4ms/step - loss: 0.1899 - accuracy: 0.9442 - val_loss: 0.5571 - val_accuracy: 0.8928
Epoch 3/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.1705 - accuracy: 0.9459 - val_loss: 0.5141 - val_accuracy: 0.8985
Epoch 4/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.1644 - accuracy: 0.9489 - val_loss: 0.6145 - val_accuracy: 0.8914
Epoch 5/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.2012 - accuracy: 0.9418 - val_loss: 0.4542 - val_accuracy: 0.8985
Epoch 6/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.1686 - accuracy: 0.9426 - val_loss: 0.5471 - val_accuracy: 0.8999
Epoch 7/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.162

In [36]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  STANDING  WALKING  WALKING_DOWNSTAIRS  \
True                                                                         
LAYING                 513        0         0        0                   0   
SITTING                  0      409        75        0                   0   
STANDING                 0      118       414        0                   0   
WALKING                  0        0         0      465                  10   
WALKING_DOWNSTAIRS       0        0         0        0                 408   
WALKING_UPSTAIRS         0        0         0        6                   8   

Pred                WALKING_UPSTAIRS  
True                                  
LAYING                            24  
SITTING                            7  
STANDING                           0  
WALKING                           21  
WALKING_DOWNSTAIRS                12  
WALKING_UPSTAIRS                 457  


In [37]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 388us/step


In [38]:
score

[0.3675457495109976, 0.9046487808227539]

- With a simple 2 layer architecture we got 90.46% accuracy and a loss of 0.36
- Since we have very little data, training a complex model is not possible, whose accuracy would have been comparable to the ML + expert engineered feature models